# Interactive Voltage and Current Display

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import tresonator as T

In [3]:
# results below obtained from previous optimization vs experimental data (62.640625 MHz)
P_in=60e3 # W
L_DUT_opt=0.0331  # m
Z_DUT_opt=0.00345  # Ohm
L_CEA_opt=0.0324  # m
Z_CEA_opt=0.0068  # Ohm
add_loss_opt=0.954  # Multiplicative coefficient to total conductivity

In [8]:
@interact(freq=(60,65,0.1))
def plot_voltage_current_vs_frequency(freq=62.640625):
    cfg_opt = T.Configuration(freq*1e6, P_in, L_DUT_opt, L_CEA_opt, 
                               Z_short_DUT = Z_DUT_opt, Z_short_CEA = Z_CEA_opt, 
                               additional_losses=add_loss_opt)

    # Calculates the voltage and current along the transmission lines
    L_CEA, L_DUT, V_CEA, V_DUT, I_CEA, I_DUT = cfg_opt.voltage_current()

    # Plotting V,I
    fig, ax = plt.subplots(2,1, sharex=True, figsize=(10,5))
    ax[0].plot(L_CEA, np.abs(V_CEA)/1e3,-L_DUT, np.abs(V_DUT)/1e3,  lw=2)
    ax[0].set_ylim(0, 45)
    ax[0].grid(True)
    ax[0].set_xlim(min(-L_DUT), max(L_CEA))
    ax[0].axvline(x=cfg_opt.L_Vprobe_CEA_fromT, ls='--', color='gray', lw=3)
    ax[0].axvline(x=-cfg_opt.L_Vprobe_DUT_fromT, ls='--', color='gray', lw=3)
    ax[0].set_ylabel('|V| [kV]', fontsize=14)
    ax[0].tick_params(labelsize=14)

    ax[1].plot(L_CEA, np.abs(I_CEA),-L_DUT, np.abs(I_DUT),  lw=2)
    ax[1].set_ylim(0, 2500)
    ax[1].grid(True)
    ax[1].axvline(x=cfg_opt.L_Vprobe_CEA_fromT, ls='--', color='gray', lw=3)
    ax[1].axvline(x=-cfg_opt.L_Vprobe_DUT_fromT, ls='--', color='gray', lw=3)
    ax[1].set_xlabel('L [m]', fontsize=14)
    ax[1].set_ylabel('|I| [A]', fontsize=14)
    ax[1].tick_params(labelsize=14)

interactive(children=(FloatSlider(value=62.640625, description='freq', max=65.0, min=60.0), Output()), _dom_cl…